# Coursera Capstone Project

##### James Fischer

This Jupyter Notebooks is used for the Coursera Capstone Project

In [70]:
import pandas as pd
import numpy as np

In [71]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto: Part 1

<b>Objective:</b> Explore and cluster the neighborhoods in Toronto
<br>
<ul>
    <li>
        <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">Wikipedia List of postal codes of Canada: M</a>
    </li>
    <li>
        This is a list of postal codes in Canada where the first letter is M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario.
    </li>
</ul>

1. Scrape the Wikipedia page and transform the data into a *pandas* dataframe

In [72]:
import requests
from bs4 import BeautifulSoup

req = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(req).text

soup = BeautifulSoup(source, 'xml')
table = soup.find('table')

col_names = ['Postal Code', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns = col_names)

for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data) == 3:
        df.loc[len(df)] = row_data

df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


2. Only process the cells that have an assigned borough. Ignore cells with a borough that is <b>Not assigned</b>.

In [73]:
df_1 = df[df.Borough != 'Not assigned'].reset_index(drop = True)
df_1.rename(columns = {'Postal Code':'Postalcode'}, inplace = True)
df_1.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [74]:
df_1.shape

(103, 3)

## Segmenting and Clustering Neighborhoods in Toronto: Part 2

Use .csv file that has the geographical coordinates of each postal code:

> `http://cocl.us/Geospatial_data`

In [75]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.rename(columns = {'Postal Code':'Postalcode'}, inplace = True)

In [76]:
geo_df.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge geographical coordinates with Toronto neighborhoods

In [77]:
df_merge = pd.merge(geo_df, df_1, on = 'Postalcode')

geo_data = df_merge[['Postalcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Segmenting and Clustering Neighborhoods in Toronto: Part 3

### Scenerio:

Your favorite basketball player, Lebron James, is playing the Toronto Raptors at Scotiabank Arena. After the game, you and your friends decide to eat some dinner. After much back and forth, you all decide on Sushi and sake bombs.

<b>Where should we go? </b>

### Import Libraries

In [78]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

# image libraries
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranform json file into pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


### Define FourSquare Credentials and Version

In [103]:
CLIENT_ID = 'OZXN5YQEZGAWVPLZUWUM1NRU1XN0XZQMXDM44IMWCGF23D3F'
CLIENT_SECRET = '055UFGKED3SR12IR5GNDPKUN5UBBULUNFGF2XJDABJL5D5SH'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OZXN5YQEZGAWVPLZUWUM1NRU1XN0XZQMXDM44IMWCGF23D3F
CLIENT_SECRET:055UFGKED3SR12IR5GNDPKUN5UBBULUNFGF2XJDABJL5D5SH


#### Define an instance of the geocoder and user_agent (foursquare_agent).

In [111]:
#Scotiabank Arena
address = '40 Bay St, Toronto, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.643429850000004 -79.37905948373245


#### Search for Sushi within 300 meters from Scotiabank Arena

In [129]:
search_query = 'Sushi Restaurant'
radius = 300
print('Search query = ' + search_query)

Search query = Sushi Restaurant


#### Define URL

In [130]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=OZXN5YQEZGAWVPLZUWUM1NRU1XN0XZQMXDM44IMWCGF23D3F&client_secret=055UFGKED3SR12IR5GNDPKUN5UBBULUNFGF2XJDABJL5D5SH&ll=43.643429850000004,-79.37905948373245&v=20180604&query=Sushi Restaurant&radius=300&limit=30'

#### Send the GET Request and examine the results

In [131]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec586c978a484001be42d4f'},
 'response': {'venues': [{'id': '5b7ae0633fcee80039f1b8ad',
    'name': 'Sushi Shop',
    'location': {'address': '65 Front St W, Union Station',
     'lat': 43.645319,
     'lng': -79.380406,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.645319,
       'lng': -79.380406}],
     'distance': 236,
     'postalCode': 'M5J 1E6',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['65 Front St W, Union Station',
      'Toronto ON M5J 1E6',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1c4941735',
      'name': 'Restaurant',
      'pluralName': 'Restaurants',
      'shortName': 'Restaurant',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1590003503',
    'hasPerk': False},
   {'id': '4be81c74ee96c92873a7fdbf',
    'name': 'Sushi Shop'

#### Transform JSON results into pandas dataframe

In [132]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,5b7ae0633fcee80039f1b8ad,Sushi Shop,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1590003503,False,"65 Front St W, Union Station",43.645319,-79.380406,"[{'label': 'display', 'lat': 43.645319, 'lng':...",236,M5J 1E6,CA,Toronto,ON,Canada,"[65 Front St W, Union Station, Toronto ON M5J ...",NaN,NaN,NaN
1,4be81c74ee96c92873a7fdbf,Sushi Shop,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1590003503,False,25 York St,43.643485,-79.380960,"[{'label': 'display', 'lat': 43.643485, 'lng':...",153,M5J 2V5,CA,Toronto,ON,Canada,"[25 York St (in Telus Tower), Toronto ON M5J 2...",in Telus Tower,NaN,NaN
2,554a62f0498e8195ef4d1841,Sushi Shop,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1590003503,False,"120 Bremner Blvd, Southcore Financial Centre",43.642570,-79.383051,"[{'label': 'display', 'lat': 43.64257, 'lng': ...",335,M5J 3A6,CA,Toronto,ON,Canada,"[120 Bremner Blvd, Southcore Financial Centre ...",Lower Simcoe,NaN,NaN
3,4b3d5882f964a520d29225e3,Oyshi Sushi,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1590003503,False,12 Queens Quay W,43.642340,-79.375853,"[{'label': 'display', 'lat': 43.64234005516476...",285,M5J 2R9,CA,Toronto,ON,Canada,"[12 Queens Quay W (at Yonge St), Toronto ON M5...",at Yonge St,NaN,NaN
4,5bc944bf033693002ca8c24d,Bento Sushi,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1590003503,False,18 York St #200,43.643162,-79.382324,"[{'label': 'display', 'lat': 43.6431619, 'lng'...",264,M5J 2T8,CA,Toronto,ON,Canada,"[18 York St #200, Toronto ON M5J 2T8, Canada]",NaN,NaN,NaN


#### Define information of interest and filter dataframe

In [141]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis = 1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.sort_values(by=['distance'])

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
1,Sushi Shop,Restaurant,25 York St,43.643485,-79.380960,"[{'label': 'display', 'lat': 43.643485, 'lng':...",153,M5J 2V5,CA,Toronto,ON,Canada,"[25 York St (in Telus Tower), Toronto ON M5J 2...",in Telus Tower,NaN,4be81c74ee96c92873a7fdbf
0,Sushi Shop,Restaurant,"65 Front St W, Union Station",43.645319,-79.380406,"[{'label': 'display', 'lat': 43.645319, 'lng':...",236,M5J 1E6,CA,Toronto,ON,Canada,"[65 Front St W, Union Station, Toronto ON M5J ...",NaN,NaN,5b7ae0633fcee80039f1b8ad
7,McDonald's,Fast Food Restaurant,"65 Front Street West, Unit 224",43.645255,-79.380570,"[{'label': 'display', 'lat': 43.64525500363231...",236,M5J 1E6,CA,Toronto,ON,Canada,"[65 Front Street West, Unit 224 (in Union Stat...",in Union Station,NaN,4b8d36f8f964a520c7ed32e3
4,Bento Sushi,Sushi Restaurant,18 York St #200,43.643162,-79.382324,"[{'label': 'display', 'lat': 43.6431619, 'lng'...",264,M5J 2T8,CA,Toronto,ON,Canada,"[18 York St #200, Toronto ON M5J 2T8, Canada]",NaN,NaN,5bc944bf033693002ca8c24d
5,Kibo Sushi House,Sushi Restaurant,16D Yonge St,43.643418,-79.375694,"[{'label': 'display', 'lat': 43.643418, 'lng':...",271,M5E 2A1,CA,Toronto,ON,Canada,"[16D Yonge St (Harbour St), Toronto ON M5E 2A1...",Harbour St,Harbourfront,5a176309dab4b138e5c06c03
3,Oyshi Sushi,Sushi Restaurant,12 Queens Quay W,43.642340,-79.375853,"[{'label': 'display', 'lat': 43.64234005516476...",285,M5J 2R9,CA,Toronto,ON,Canada,"[12 Queens Quay W (at Yonge St), Toronto ON M5...",at Yonge St,NaN,4b3d5882f964a520d29225e3
10,Epic,Wine Bar,100 Front Street West,43.645515,-79.381239,"[{'label': 'display', 'lat': 43.64551466995504...",290,M5J 1E3,CA,Toronto,ON,Canada,"[100 Front Street West, Toronto ON M5J 1E3, Ca...",NaN,NaN,4ad4c05df964a5203af620e3
9,ResoSolutions - Online Restaurant Booking Syst...,Restaurant,1 Yonge Street,43.642566,-79.375255,"[{'label': 'display', 'lat': 43.64256622923830...",321,M5E 1W7,CA,Toronto,ON,Canada,"[1 Yonge Street (Suite 1801), Toronto ON M5E 1...",Suite 1801,NaN,55413a52498e17074d38e6f5
8,Bottom Line Restaurant & Bar,Bar,22 Front St W,43.646286,-79.378086,"[{'label': 'display', 'lat': 43.646286, 'lng':...",327,M5J 2T3,CA,Toronto,ON,Canada,"[22 Front St W, Toronto ON M5J 2T3, Canada]",NaN,NaN,4d542fc4f9f9b60c3b8d2be6
2,Sushi Shop,Restaurant,"120 Bremner Blvd, Southcore Financial Centre",43.642570,-79.383051,"[{'label': 'display', 'lat': 43.64257, 'lng': ...",335,M5J 3A6,CA,Toronto,ON,Canada,"[120 Bremner Blvd, Southcore Financial Centre ...",Lower Simcoe,NaN,554a62f0498e8195ef4d1841


#### Display Sushi Restaurants within 300 meteres of Scotiabank Arena

In [139]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent Scotiabank Arena
folium.features.CircleMarker(
    [latitude, longitude],
    radius = 8,
    color = 'red',
    popup = 'Scotiabank Arena',
    fill = True,
    fill_color = 'red',
    fill_opacity = 1
).add_to(venues_map)

# add the Sushi restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius = 5,
        color = 'blue',
        popup = label,
        fill = True,
        fill_color = 'blue',
        fill_opacity = 1
    ).add_to(venues_map)

# display map
venues_map

#### Explore the closest Shshi restaurant -- _Sushi Shop_

In [142]:
venue_id = '4be81c74ee96c92873a7fdbf' # ID of Sushi Shop
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4be81c74ee96c92873a7fdbf?client_id=OZXN5YQEZGAWVPLZUWUM1NRU1XN0XZQMXDM44IMWCGF23D3F&client_secret=055UFGKED3SR12IR5GNDPKUN5UBBULUNFGF2XJDABJL5D5SH&v=20180604'

#### Send GET request for result

In [143]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4be81c74ee96c92873a7fdbf',
 'name': 'Sushi Shop',
 'contact': {'phone': '4162039388',
  'formattedPhone': '(416) 203-9388',
  'facebook': '217737477562',
  'facebookUsername': 'sushishopboutique',
  'facebookName': 'Sushi Shop'},
 'location': {'address': '25 York St',
  'crossStreet': 'in Telus Tower',
  'lat': 43.643485,
  'lng': -79.38096,
  'labeledLatLngs': [{'label': 'display', 'lat': 43.643485, 'lng': -79.38096}],
  'postalCode': 'M5J 2V5',
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['25 York St (in Telus Tower)',
   'Toronto ON M5J 2V5',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/sushi-shop/4be81c74ee96c92873a7fdbf',
 'categories': [{'id': '4bf58dd8d48988d1c4941735',
   'name': 'Restaurant',
   'pluralName': 'Restaurants',
   'shortName': 'Restaurant',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d1d2941

#### Get Sushi Shop's overall rating

In [144]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.1


#### Get the number of tips

In [145]:
result['response']['venue']['tips']['count']

8

Create URL and send GET request (limit 10)

In [149]:
limit = 10
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(
    venue_id, 
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec58ac37828ae00285b56fc'},
 'response': {'tips': {'count': 8,
   'items': [{'id': '5ac550fe8c812a099f09b1bb',
     'createdAt': 1522880766,
     'text': 'The sushi tastes great, the are so many options, but what really stands out are the staff. So helpful and incredibly friendly. Makes the experience great. I look forward to going there everytime.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ac550fe8c812a099f09b1bb',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 1,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '246118',
      'firstName': 'Nihir',
      'lastName': 'M',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/246118-IZGD012AZOHGJTER.jpg'}},
     'authorInteractionType': 'liked'}]}}}

## After eating dinner at Sushi Shop, you discuss what to do next

Latitude and longitude of Sushi Shop

In [153]:
latitude = 43.643485
longitude = -79.380960

In [154]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=OZXN5YQEZGAWVPLZUWUM1NRU1XN0XZQMXDM44IMWCGF23D3F&client_secret=055UFGKED3SR12IR5GNDPKUN5UBBULUNFGF2XJDABJL5D5SH&ll=43.643485,-79.38096&v=20180604&radius=300&limit=30'

#### Send GET request and examine results

In [155]:
import requests
results = requests.get(url).json()
'There are {} around Sushi Shop restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Sushi Shop restaurant.'

#### Get relevant part of JSON

In [156]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '55561f8a498e2e5c866ac4c9',
  'name': 'Union Pearson Express',
  'location': {'address': '61 Front St. W',
   'lat': 43.64436200658875,
   'lng': -79.38319927907953,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.64436200658875,
     'lng': -79.38319927907953}],
   'distance': 205,
   'postalCode': 'M5J 2Z8',
   'cc': 'CA',
   'neighborhood': 'Entertainment District',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['61 Front St. W', 'Toronto ON M5J 2Z8', 'Canada']},
  'categories': [{'id': '4bf58dd8d48988d129951735',
    'name': 'Train Station',
    'pluralName': 'Train Stations',
    'shortName': 'Train Station',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups'

#### Process JSON and convert it to a clean dataframe

In [159]:
dataframe = pd.json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(20)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,Union Pearson Express,Train Station,61 Front St. W,43.644362,-79.383199,"[{'label': 'display', 'lat': 43.64436200658875, 'lng': -79.38319927907953}]",205,M5J 2Z8,CA,Entertainment District,Toronto,ON,Canada,"[61 Front St. W, Toronto ON M5J 2Z8, Canada]",NaN,55561f8a498e2e5c866ac4c9
1,Real Sports Apparel,Sporting Goods Shop,15 York St. Unit B,43.642860,-79.380184,"[{'label': 'display', 'lat': 43.64285984835777, 'lng': -79.38018363195047}]",93,M5J,CA,NaN,Toronto,ON,Canada,"[15 York St. Unit B (Maple Leaf Square), Toronto ON M5J, Canada]",Maple Leaf Square,4be46c832468c92887c1fe42
2,Maple Leaf Square,Plaza,15 York St.,43.642925,-79.380892,"[{'label': 'display', 'lat': 43.64292522840183, 'lng': -79.38089182467687}]",62,NaN,CA,NaN,Toronto,ON,Canada,"[15 York St. (Bremner Blvd.), Toronto ON, Canada]",Bremner Blvd.,4bdb8c1cc79cc928a77583e9
3,WVRST,Beer Bar,65 Front St W,43.644968,-79.381376,"[{'label': 'display', 'lat': 43.64496809087762, 'lng': -79.38137631843854}]",168,M5J 1E6,CA,NaN,Toronto,ON,Canada,"[65 Front St W, Toronto ON M5J 1E6, Canada]",NaN,5c08474447f876002c736b68
4,Scotiabank Arena,Basketball Stadium,40 Bay St,43.643446,-79.379040,"[{'label': 'display', 'lat': 43.64344617535107, 'lng': -79.37903988889991}]",154,M5J 2X2,CA,NaN,Toronto,ON,Canada,"[40 Bay St, Toronto ON M5J 2X2, Canada]",NaN,4b155081f964a520b4b023e3
5,iQ Food Co,Salad Place,18 York Street,43.642851,-79.382081,"[{'label': 'display', 'lat': 43.642851, 'lng': -79.382081}]",114,M5J 0B2,CA,NaN,Toronto,ON,Canada,"[18 York Street (Bremner Ave), Toronto ON M5J 0B2, Canada]",Bremner Ave,5346c98a498ed612110d0f60
6,Air Canada Club,Lounge,Air Canada Centre,43.643224,-79.379159,"[{'label': 'display', 'lat': 43.643224100030416, 'lng': -79.37915938219528}]",147,M5J 2L2,CA,NaN,Toronto,ON,Canada,"[Air Canada Centre, Toronto ON M5J 2L2, Canada]",NaN,4b60d5b9f964a52071fc29e3
7,Le Germain Hotel,Hotel,75 Bremner Boulevard,43.643125,-79.380918,"[{'label': 'display', 'lat': 43.64312502979691, 'lng': -79.38091770864038}]",40,M5J 0A1,CA,NaN,Toronto,ON,Canada,"[75 Bremner Boulevard (York Street), Toronto ON M5J 0A1, Canada]",York Street,4cb87a334c60a093c04a39ca
8,Pilot Coffee Roasters,Coffee Shop,65 Front St W,43.645018,-79.380415,"[{'label': 'display', 'lat': 43.64501814464698, 'lng': -79.3804150931199}]",176,M5J 1E6,CA,NaN,Toronto,ON,Canada,"[65 Front St W (btwn Bay St & York St), Toronto ON M5J 1E6, Canada]",btwn Bay St & York St,563d2f2dcd10bcf27ae37c3b
9,Delta Hotels by Marriott Toronto,Hotel,75 Lower Simcoe Street,43.642882,-79.383949,"[{'label': 'display', 'lat': 43.6428819, 'lng': -79.3839491}]",249,M5J 3A6,CA,NaN,Toronto,ON,Canada,"[75 Lower Simcoe Street, Toronto ON M5J 3A6, Canada]",NaN,53357710498e20817350cfb4


#### Visualize these items on the map around our location

In [158]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15)


# add Sushi Shop as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius = 8,
    color = 'red',
    popup = 'Sushi Shop',
    fill = True,
    fill_color = 'red',
    fill_opacity = 1
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius = 5,
        color = 'blue',
        popup = label,
        fill = True,
        fill_color = 'blue',
        fill_opacity = 1
        ).add_to(venues_map)

# display map
venues_map

Looks like we're headed to <b>WVRST Beer Bar</b>!